In [1]:
import numpy as np
import matplotlib.pyplot as plt

from packet_utils import compute_crc, ble_whitening

# payload -> packet (bytes)(whitened) DONE
# bytes -> bits
# bits -> IQ

In [2]:
def create_packet(payload: np.ndarray, base_addr: int) -> np.ndarray:
    """
    Create a physical BLE packet from payload and base address.

    Packet Structure:
    ┌───────────┬──────────────┬───────────────┬───────────┬────────┬────────┬─────────┐
    │ Preamble  │ Base Address │ Prefix (0x00) │ S0 (0x00) │ Length │ PDU    │ CRC     │
    ├───────────┼──────────────┼───────────────┼───────────┼────────┼────────┼─────────┤
    │ 0xAA/0x55 │ 4 bytes      │ 1 byte        │ 1 byte    │ 1 byte │ 0–255B │ 3 bytes │
    └───────────┴──────────────┴───────────────┴───────────┴────────┴────────┴─────────┘
    """
    # Set the preamble based on the LSB of the base address
    preamble = np.uint8(0x55 if (base_addr & 0x01) else 0xAA)

    # Convert base address into 4 bytes in little-endian order
    base_addr_len = 4
    base_addr_bytes = np.array([np.uint8((base_addr >> (i * 8)) & 0xFF) for i in range(base_addr_len)], dtype=np.uint8)

    # Set prefix, S0 and length bytes
    prefix = np.uint8(0x00)
    s0 = np.uint8(0x00)
    length = np.uint8(len(payload))

    # Append CRC
    ready_for_crc = np.concatenate(([s0, length], payload))
    crc = compute_crc(ready_for_crc, crc_init=0x00FFFF, crc_poly=0x00065B, crc_size=3)
    ready_for_whitening = np.concatenate((ready_for_crc, crc))

    # Whiten from S0 to CRC
    whitened, _ = ble_whitening(ready_for_whitening)
    packet = np.concatenate(([preamble], base_addr_bytes, [prefix], whitened))

    return packet


# Example values:
base_addr = 0x12345678
payload = np.array([0xAB, 0xCD], dtype=np.uint8)

packet = create_packet(payload, base_addr)
print("Packet bytes:")
print([hex(byte) for byte in packet])

Packet bytes:
['0xaa', '0x78', '0x56', '0x34', '0x12', '0x0', '0x40', '0xb0', '0x17', '0xe', '0x70', '0x36', '0x9d']
